In [128]:
import os
import pandas as pd

In [129]:
NAME = '薛少栋'
YEAR = 2022
MONTH = 12
USEFUL_COLS = ['编号', '任务名称', '总计消耗', '最后修改日期']
CLIENTS = [
    '北大AI', '大唐', '东软', '二郎神视觉', 
    '辉曦智能', '经纬恒润', ('美团', '三快科技'), '纽劢', '启林', 
    '识渊', '中石油', '纵目'
]

In [130]:
def find_client(title):
    result = None
    for c in CLIENTS:
        if type(c) == str:
            if c in title:
                result = c
        else:
            for part_c in c:
                if part_c in title:
                    result = c[0]
    return result

In [131]:
def fill_unsure(raw_df, fill_list, col_num=len(USEFUL_COLS)):
    locs = raw_df[raw_df.isna().any(axis=1)].index
    for i in range(len(fill_list)):
        raw_df.iloc[locs[i], col_num] = fill_list[i]

In [132]:
def list_dir(cur_path: str):
    """
    列举文件根目录下各文件路径的函数。
    :param cur_path: 根目录。
    :return: 根目录下所有子目录里的所有文件。
    """
    file_paths = []
    for root, dirs, files in os.walk(cur_path):
        for file in files:
            file_paths.append(os.path.join(root, file))
    return file_paths

In [133]:
works = list_dir('2022_12')
total_df = pd.concat([pd.read_csv(w, index_col=False)[:-1] for w in works])[USEFUL_COLS]
total_df['最后修改日期'] = pd.to_datetime(total_df['最后修改日期'])
monthly_df = total_df[(total_df['最后修改日期'].dt.year == YEAR) & (total_df['最后修改日期'].dt.month == MONTH)].copy()
monthly_df['客户'] = monthly_df['任务名称'].map(find_client)
monthly_df[monthly_df['客户'].isna()]

,编号,任务名称,总计消耗,最后修改日期,客户
11,8880,>线下通错软件有新需求,2.0,2022-12-02,None


In [134]:
fill_unsure(monthly_df, ['识渊'])

In [141]:
groupby_df = monthly_df.groupby(by='客户').agg({'总计消耗': 'sum', '编号': 'count', '最后修改日期': 'count'})
groupby_df.rename(columns={'总计消耗': '总工时', '编号': '任务数', '最后修改日期': 'temp'}, inplace=True)
merge_df = pd.merge(left=monthly_df, right=groupby_df, on='客户')
merge_df.drop(columns=['编号', '总计消耗', '最后修改日期'], inplace=True)
merge_df.set_index(keys=['客户', '任务数', '总工时', 'temp'], inplace=True)

In [137]:
merge_df.to_excel(f'result/{NAME}{YEAR}-{MONTH}任务1.xlsx')